In [1]:
import numpy as np
import scipy.signal
import scipy.io
import matplotlib.pyplot as plt
import math
import scipy.optimize
import pandas as pd
from IPython.display import Audio
import csv

In [2]:
def nextpow2(N):
    n = 1
    while n < N: n *= 2
    return n

In [3]:
def F_getF0FromSpectre(
    fftFreq_hz_v, fftAmpl_v, H, fmin_hz, fmax_hz, Nfft, sr_hz, plot=False
):
    """
    inputs:
        - fftFreq_hz_v (N/2+1,): vector containing the FFT frequencies in Hz
        - fftAmpl_v (N/2+1,): vector containing the FFT amplitude
        - H: number of times the spectrum is decimated
        - fmin_hz: minimum frequency in Hz to look for F0
        - fmax_hz: maximum frequency in Hz to look for F0
        - Nfft: fft size
        - sr_hz: sampling rate
    outputs:
        - spFreq_hz_v: vector containing the SpectralProduct frequencies in Hz
        - spAmpl_v: vector containing the SpectralProduct amplitudes
        - f0_hz: estimated F0 in Hz
    """
    R = int(Nfft / (2 * H) + 1)
    P = np.zeros(R)
    h = np.atleast_2d(np.arange(1, H + 1)).T
    f_k = np.arange(1, R + 1)
    A = np.abs(fftAmpl_v[h * f_k])
    P = np.prod(A, axis=0)
    F = np.linspace(0, sr_hz / (2 * H), R)

    spAmpl_v = P
    spFreq_hz_v = F

    n_min = int(fmin_hz / sr_hz * Nfft)
    n_max = int(fmax_hz / sr_hz * Nfft)
    assert n_max < R, "f_max est trop grand !"

    f0_hz = F[n_min:n_max][np.argmax(spAmpl_v[n_min:n_max])]

    return spFreq_hz_v, spAmpl_v, f0_hz

In [80]:
def F_getHarmonicsFromF0AndSpectre(
    fftFreq_hz_v, fftAmpl_v, f0_hz, Nfft, sr_hz, alpha=0.01, beta=0.0002, plot=False
):
    """
    inputs:
        - fftFreq_hz_v (N/2+1,): vector containing the FFT frequencies in Hz
        - fftAmpl_v (N/2+1,): vector containing the FFT amplitude
        - f0_hz: estimated F0 in Hz
        - Nfft: fft size
        - sr_hz: sampling rate
    outputs:
        - harmoFreq_k_v: vector containing the position/index of the Harmonics (position in fftFreq_hz_v or fftAmpl_v)
        - harmoAmpl_v:  vector containing the Harmonic amplitudes
    """
    def get_max_harmo(h):
        f_inharmo = h * f0_hz * np.sqrt(1 + (h**2 - 1) * beta)
        f_kmin = (1 - alpha) * f_inharmo
        f_kmax = (1 + alpha) * f_inharmo

        n_min = np.int64(f_kmin / sr_hz * Nfft)
        n_max = np.int64(f_kmax / sr_hz * Nfft)

        i_harmo = np.argmax(fftAmpl_v[n_min:n_max])
        a_harmo = fftAmpl_v[n_min:n_max][i_harmo]

        return i_harmo + n_min, a_harmo

    # f0_hz = fftFreq_hz_v[get_max_harmo(1)[0]]
    # print("ajusté à", f0_hz)
    nHarmos = 10  # TODO: calculer le bon nombre d'harmoniques

    harmoFreq_k_v = np.zeros(nHarmos, np.int64)
    harmoAmpl_v = np.zeros(nHarmos)
    for h in range(nHarmos):
        harmoFreq_k_v[h], harmoAmpl_v[h] = get_max_harmo(h + 1)

    return harmoFreq_k_v, harmoAmpl_v

In [81]:
# note <-> fréquence
def freq_to_midi(freq: float):
    note_number_f = 12 * math.log2(freq / 440) + 49
    note_number = round(note_number_f)
    delta = note_number_f - note_number
    return note_number, delta


def midi_to_name(midi: int):
    notes = ["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"]
    note = (midi - 1) % len(notes)
    note = notes[note]
    octave = (midi + 8) // len(notes)

    return note + str(octave)

In [82]:
def get_envelope(t, x, d=1):
    def hl_envelopes_idx(s, dmin=1, dmax=1, split=False):
        """
        Input :
        s: 1d-array, data signal from which to extract high and low envelopes
        dmin, dmax: int, optional, size of chunks, use this if the size of the input signal is too big
        split: bool, optional, if True, split the signal in half along its mean, might help to generate the envelope in some cases
        Output :
        lmin,lmax : high/low envelope idx of input signal s
        """

        # locals min      
        lmin = (np.diff(np.sign(np.diff(s))) > 0).nonzero()[0] + 1 
        # locals max
        lmax = (np.diff(np.sign(np.diff(s))) < 0).nonzero()[0] + 1 
        
        if split:
            # s_mid is zero if s centered around x-axis or more generally mean of signal
            s_mid = np.mean(s) 
            # pre-sorting of locals min based on relative position with respect to s_mid 
            lmin = lmin[s[lmin]<s_mid]
            # pre-sorting of local max based on relative position with respect to s_mid 
            lmax = lmax[s[lmax]>s_mid]

        # global min of dmin-chunks of locals min 
        lmin = lmin[[i+np.argmin(s[lmin[i:i+dmin]]) for i in range(0,len(lmin),dmin)]]
        # global max of dmax-chunks of locals max 
        lmax = lmax[[i+np.argmax(s[lmax[i:i+dmax]]) for i in range(0,len(lmax),dmax)]]
        
        return lmin,lmax
    _, lmax = hl_envelopes_idx(x, dmax=d)
    lmax = np.interp(t, t[lmax], x[lmax])
    return lmax

In [180]:
def traiter(filename, f0, harmos, titre):
    plt.ioff()
    window = np.hamming

    fs, sig = scipy.io.wavfile.read(filename)
    sig = sig.astype(np.float32)
    sig/=np.max(abs(sig))

    t = np.arange(len(sig)) / fs

    env = get_envelope(t, sig, 100)

    plt.figure(figsize=(15,3))
    plt.title("Enveloppe de "+titre)
    plt.plot(t, sig)
    plt.plot(t, env)
    plt.savefig(filename+".env.png")

    ##### CUTTING

    MARGE_START_BEFORE = int(0.01 * fs)
    MARGE_START_AFTER = int(0.1 * fs)
    MARGE_SEARCH_END = int(0.2 * fs)
    MARGE_SILENCE = 0.01

    i_silence = np.argmax(abs(sig)) - MARGE_START_BEFORE
    print(f"le signal commence à {i_silence/fs} s")
    silence_level = np.max(abs(sig[:i_silence]))
    print(f"niveau de silence: {20*np.log10(silence_level):.2f} dB ({silence_level:.2f} V)")

    i_start = i_silence + MARGE_START_AFTER

    i_searchend = i_start + MARGE_SEARCH_END
    i_end = i_searchend + np.argmax(env[i_searchend:] < silence_level + MARGE_SILENCE)
    print("le signal finit à", i_end / fs)

    zoom = 0.01
    plt.suptitle("Découpe de " + titre)
    plt.figure(figsize=(15, 5))
    plt.title(f"bruit de fond: {20*np.log10(silence_level):.2f} dB")
    plt.subplot(1, 3, 1)
    plt.plot(t, abs(sig))
    plt.plot(t, env)
    plt.hlines(silence_level, t[0], t[-1], "red")
    plt.vlines(t[i_silence], 0, 1, "red")
    plt.xlim(t[i_silence] - zoom, t[i_silence] + zoom)
    plt.subplot(1, 3, 2)
    plt.plot(t, abs(sig))
    plt.plot(t, env)
    plt.hlines(silence_level, t[0], t[-1], "red")
    plt.vlines(t[i_start], 0, 1, "red")
    plt.xlim(t[i_start] - zoom, t[i_start] + zoom)
    plt.subplot(1, 3, 3)
    plt.plot(t, abs(sig))
    plt.plot(t, env)
    plt.hlines(silence_level, t[0], t[-1], "red")
    plt.vlines(t[i_end], 0, 1, "red")
    plt.xlim(t[i_end] - zoom, t[i_end] + zoom)
    plt.savefig(filename+".decoupe.png")

    sig_cut = sig[i_start:i_end]
    t_cut = t[i_start:i_end]
    plt.figure(figsize=(15, 3))
    plt.title("Régime stationnaire de " + titre)
    plt.plot(t_cut, sig_cut)
    plt.savefig(filename+".statio.png")

    ####
    Nfft = 1 * nextpow2(sig_cut.shape[0])
    freqs = np.fft.fftfreq(Nfft, 1/fs)
    fft = np.fft.fft(sig_cut * window(len(sig_cut)), Nfft) / len(sig_cut)
    mag = abs(fft)

    # detection f0
    # if pd.isna(f0):
    #     f0_min = 100
    #     f0_max = 1000
    # else:
    #     f0_min = f0 - 50
    #     f0_max = f0 + 50
    # _, _, f0 = F_getF0FromSpectre(freqs, mag, 2, f0_min, f0_max, Nfft, fs)

    midi, delta = freq_to_midi(f0)
    print(f"{f0:.2f} Hz, (={midi_to_name(midi)}{delta:+.2f} Hz)")

    if len(harmos)==0:
        i_harmos, _ = F_getHarmonicsFromF0AndSpectre(freqs, mag, f0, Nfft, fs)
        harmos = freqs[i_harmos]

    harmos = [f0] + harmos
    harmos = np.array(harmos)
    plt.figure(figsize=(15,3))
    plt.title("Spectre de " + titre)
    plt.xlim(min(harmos)-200,max(harmos)+200)
    plt.plot(freqs, 20*np.log10(mag))
    for f in harmos:
        plt.axvline(f, linestyle="--", color = "red")
        # plt.text(f, 0, f"{f:.1f} Hz")
        plt.annotate(f"{f:.1f} Hz", (f, mag[np.argmin(freqs-f > 0)]))
    plt.savefig(filename+".spectre.png")

    rapport_harmo = harmos / harmos[0]

    pd.DataFrame(np.c_[harmos, rapport_harmo], columns=["Fréquence", "Rapport harmonique"])

    ####
    plt.figure(figsize=(15, 3*len(harmos)))
    plt.suptitle("Analyse des partiels de " + titre)
    for i, f in enumerate(harmos):
        plt.subplot(len(harmos), 1, i+1)
        delta = 1
        window = np.where(np.logical_and(freqs > f-delta, freqs < f+delta), 1, 0)
        filt = scipy.signal.butter(
            2, (f/delta, f*delta), btype="bandpass", fs=fs, output="sos"
        )
        plt.loglog(*scipy.signal.sosfreqz(filt,fs=fs))
        plt.show()
        sig_filtered = scipy.signal.sosfilt(filt, sig_cut)
        env_filtered = get_envelope(t_cut, sig_filtered, 5)
        plt.title(f"Partiel $f_{i}={f:.1f} Hz = {f/harmos[0]:.1f} f_0$")
        plt.plot(t_cut, sig_filtered, label="partiel filtré")
        plt.plot(t_cut, env_filtered, label="enveloppe")

        def exp_decay(x, A, alpha):
            return A * np.exp(-alpha * x)
        try:
            popt, pcov = scipy.optimize.curve_fit(exp_decay, t_cut, env_filtered)
            perr = np.sqrt(np.diag(pcov))
            print(f, popt, perr)
            plt.plot(t_cut, exp_decay(t_cut, *popt), label="enveloppe fittée")
        except RuntimeError:
            print(f"pas pu fitter pour le f_{i}")
        # display(Audio(sig_filtered, rate=fs))
    plt.legend()
    plt.savefig(filename+".partiels.png")
    plt.ion()




In [181]:
files = pd.read_csv("harmoniques.csv")
files

,filename,titre,f0 theorique,f1,f2,f3,f4,f5,f6,f7
0,glock-a5.wav,NaN,880.0,2440.0,3734.0,4664.0,7490.0,7674.0,2598.0,NaN
1,glock-a6.wav,NaN,1763.0,3571.0,4836.0,9167.0,10738.0,NaN,NaN,NaN
2,glock-a7.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,glock-c6.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,glock-c7.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,glock-c8.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,glock-f5.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,glock-f6.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,glock-f7.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,timbale-aigu-1.wav,Timbale Aigu 1/4,125.0,188.0,249.0,269.0,309.0,307.0,NaN,NaN


In [182]:
from tqdm import tqdm
for _, file in tqdm(files.iterrows()):
    filename = file["filename"]
    titre = file["titre"]
    titre = filename.split(".")[0] if pd.isna(titre) else titre
    f0 = file["f0 theorique"]
    harmos = [file[k] for k in file.keys()[3:] if not pd.isna(file[k])]
    harmos = sorted(harmos)
    print(harmos)
    traiter(filename, f0, harmos, titre)

0it [00:00, ?it/s]C:\Users\etian\AppData\Local\Temp\ipykernel_7136\1516058765.py:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, sig = scipy.io.wavfile.read(filename)


[2440.0, 2598.0, 3734.0, 4664.0, 7490.0, 7674.0]
le signal commence à 2.8608125 s
niveau de silence: -50.18 dB (0.00 V)
le signal finit à 4.406208333333334
880.00 Hz, (=A5+0.00 Hz)
880.0 [0.25676588 0.69388039] [0.00056108 0.00062839]
2440.0 [1057.02756329    4.0324274 ] [1.03265198e+01 3.16452223e-03]
2598.0 [489.14180965   4.00166867] [4.07860893e+00 2.70015633e-03]
3734.0 [3.21874644e+08 8.54640282e+00] [1.00054000e+07 1.02947789e-02]
4664.0 [2.8468057e+09 8.3234402e+00] [5.77169560e+07 6.71099592e-03]
7490.0 [1.86596668e+05 5.52820231e+00] [7.78823250e+03 1.36770629e-02]
7674.0 [3.66952285e+05 5.75222242e+00] [1.60867298e+04 1.43810426e-02]


1it [00:04,  4.38s/it]

[3571.0, 4836.0, 9167.0, 10738.0]
le signal commence à 2.4133541666666667 s
niveau de silence: -50.51 dB (0.00 V)
le signal finit à 3.170125
1763.00 Hz, (=A6+0.03 Hz)
1763.0 [5.1733628  1.79327447] [0.03887699 0.00275422]


C:\Users\etian\AppData\Local\Temp\ipykernel_7136\1516058765.py:92: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15,3))


3571.0 [6.60007806 3.71352463] [0.2820626  0.01615511]
4836.0 [2.18784983e+05 6.70306495e+00] [4.63399341e+03 8.18178830e-03]
pas pu fitter pour le f_3
10738.0 [2.26662338e+09 1.10715619e+01] [2.13563491e+08 3.68210803e-02]


2it [00:07,  3.77s/it]

[]
le signal commence à 2.3794583333333335 s
niveau de silence: -38.53 dB (0.01 V)
le signal finit à 3.0151041666666667


2it [00:09,  4.53s/it]


ValueError: cannot convert float NaN to integer